In [ ]:
%config IPCompleter.greedy=True

In [1]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [3]:
local_zip = '/tmp/horse-or-human.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human/')
zip_ref.close()

In [4]:
import os
import shutil

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

In [5]:
def create_dir(file_dir):
    if os.path.exists(file_dir):
        print('true')
        #os.rmdir(file_dir)
        shutil.rmtree(file_dir)#删除再建立
        os.makedirs(file_dir)
    else:
        os.makedirs(file_dir)

In [14]:
HORSE_SOURCE_DIR = "/tmp/horse-or-human/horses/"
TRAINING_HORSE_DIR = "/tmp/horse-v-humans/training/horses/"
TESTING_HORSE_DIR = "/tmp/horse-v-humans/testing/horses/"
HUMAN_SOURCE_DIR = "/tmp/horse-or-human/humans/"
TRAINING_HUMANS_DIR = "/tmp/horse-v-humans/training/humans/"
TESTING_HUMANS_DIR = "/tmp/horse-v-humans/testing/humans/"

create_dir(TRAINING_HORSE_DIR)
create_dir(TESTING_HORSE_DIR)
create_dir(TRAINING_HUMANS_DIR)
create_dir(TESTING_HUMANS_DIR)

In [15]:
split_data(HORSE_SOURCE_DIR, TRAINING_HORSE_DIR, TESTING_HORSE_DIR, 0.9)
split_data(HUMAN_SOURCE_DIR, TRAINING_HUMANS_DIR, TESTING_HUMANS_DIR, 0.9)

In [16]:
print(len(os.listdir(TRAINING_HORSE_DIR)))
print(len(os.listdir(TESTING_HORSE_DIR)))
print(len(os.listdir(TRAINING_HUMANS_DIR)))
print(len(os.listdir(TESTING_HUMANS_DIR)))

450
50
474
53


In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

2022-04-06 23:02:53.957464: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 23:02:53.958053: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.


In [20]:

TRAINING_DIR = "/tmp/horse-v-humans/training"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=100, class_mode='binary', target_size=(150, 150))

TESTING_DIR = "/tmp/horse-v-humans/testing"
testing_datagen = ImageDataGenerator(rescale=1.0/255.0)
testing_generator = testing_datagen.flow_from_directory(TESTING_DIR, batch_size=100, class_mode='binary', target_size=(150, 150))

Found 924 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [21]:
history = model.fit_generator(train_generator, epochs=2, verbose=1, validation_data=testing_generator)

Epoch 1/2
10/10 [==============================] - 27s 3s/step - loss: 0.4239 - acc: 0.8463 - val_loss: 0.1592 - val_acc: 0.9709
Epoch 2/2
10/10 [==============================] - 27s 3s/step - loss: 0.0893 - acc: 0.9697 - val_loss: 0.0529 - val_acc: 0.9806


In [26]:
history.history['val_acc']

[0.9708738, 0.98058254]